In [20]:
%config IPCompleter.greedy=True

In [21]:
#adhere to https://github.com/tensorflow/tensorflow/blob/r0.7/tensorflow/examples/tutorials/mnist/input_data.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import csv

data_path = "./weapon_data.csv"

In [22]:
def getCsvHeader(filename):
    with open(filename, mode='r') as f:
        reader = csv.reader(f)
        header = next(reader)
        return header
    
def getCsvAsDict(filename):
    header = getCsvHeader(filename)
    result = { h: [] for h in header }
    with open(filename, mode='r') as file:
        for row in csv.DictReader(file):
            for h in header:
                result[h].append(row[h])

    return result

In [23]:
features = getCsvAsDict(data_path)
print(len(features['weapshowname']))
#print(features)

148


In [31]:
def tensorToParameterDict(tensor, cols_to_vars_dict):
    idx =  0 #keep track of the id to find the right value in the processed tensor
    result = dict()
    for key in cols_to_vars_dict:
        '''
        e.g., key = _NumericColumn(key='bdrop', shape=(1,), default_value=None, 
                    dtype=tf.float32, normalizer_fn=None)
        then key[0] = 'bdrop'
        '''
        if key[0] in NUMERICAL_PARAMS:
            result[key[0]] = str(tensor[idx])
        
            '''
            e.g., key = _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(
                        key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 
                        'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0))
            then key[0] = _VocabularyListCategoricalColumn(
                        key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 
                        'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0)
            and key[0][0] = 'firemode'
            and key[0][0][0] = 'Automatic'
            '''
        elif (len(key[0]) > 0) and key[0][0] in CATEGORICAL_PARAMS:
            for i in range(0, len(key[0][1])-1):
                param = key[0][0] + "_" + key[0][1][i]
                result[param] = str(tensor[idx])
                idx += 1
                
        idx += 1
    return result

In [37]:
#based on https://www.tensorflow.org/get_started/feature_columns

#define constants for data processing
CATEGORICAL_PARAMS  = ['type', 'firemode'] #exclude ammo because I'm not interested in decoding its values
NUMERICAL_PARAMS = ['rof', 'damages_first', 'damages_last', 'dmg_distances_first', 'dmg_distances_last', 'initialspeed',
                    'drag', 'magsize', 'shotspershell', 'reloadempty', 'hordispersion', 'verdispersion', 
                    'hiprecoildec', 'hiprecoilright', 'hiprecoilup' ]
WEAPON_TYPES = ['Shotgun', 'Pistol', 'Rifle', 'Submachine Gun', 'Sniper Rifle', 'Light Machine Gun']
WEAPON_FIREMODES = ['Automatic', 'Semi-Automatic', 'Single-Action']

#convert all numerical data to float so they can be used as 'tf.feature_column.numeric_column'
for key, values in features.items():
    if key in NUMERICAL_PARAMS:
        features[key] = [float('{:.4f}'.format(float(value))) for value in values]

#one hot encoding of categorical data
type_column = tf.feature_column.categorical_column_with_vocabulary_list(key='type', vocabulary_list=WEAPON_TYPES)
type_column = tf.feature_column.indicator_column(type_column) #could use embedding column to further reduce dimensions

#one hot encoding of categorical data
firemode_colum = tf.feature_column.categorical_column_with_vocabulary_list(key='firemode', vocabulary_list=WEAPON_FIREMODES)
firemode_colum = tf.feature_column.indicator_column(firemode_colum) #could use embedding column to further reduce dimensions

#create this bucket just because i don't wnat to define the vocabulary list with every ammo name
ammo_column = tf.feature_column.categorical_column_with_hash_bucket(key = "ammo", hash_bucket_size = 31) 
#actual dimension reduction with an embedding column
ammo_column = tf.feature_column.embedding_column(ammo_column, dimension=1)

#create list with all feature columns
columns = [tf.feature_column.numeric_column(param) for param in NUMERICAL_PARAMS]
columns.append(type_column)
columns.append(firemode_colum)
columns.append(ammo_column)

#print(columns)

#make sure you know whats going on inside
cols_to_vars_dict = {}

#create an input layer with your feature columns and make sure you get the cols_to_vars dictionary!
inputs = tf.feature_column.input_layer(features, columns, cols_to_vars=cols_to_vars_dict, trainable = False)
#print(cols_to_vars_dict)
#print(inputs)

var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer() #needed for the feature columns

weapon_data = [];

#shuffle the whole input layer
tf.set_random_seed(19071991)
with tf.Session() as sess:
    sess.run((var_init, table_init))
    weapon_data = sess.run(tf.random_shuffle(inputs))
    print("Shuffled %i" %len(weapon_data) + " input data")


Shuffled 148 input data


In [39]:
genDict = {}

def addGeneratedWeapon(generatedWeaponTensorDict, dictToAdd):
    for key, value in generatedWeaponTensorDict.items():
        if key not in dictToAdd:
            dictToAdd[key] = []
        dictToAdd[key].append(value)
        
def debug_printDict(dictionary):
    for key, value in dictionary.items():
        print(key, "=", value)

var_init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(var_init)
    
    batch = weapon_data[0:10]
    #print(batch)
    
    
    tensorDict = [tensorToParameterDict(b, cols_to_vars_dict) for b in batch]
    [addGeneratedWeapon(t, genDict) for t in tensorDict]
    #debug_printDict(tensorDict[0])
    
    
debug_printDict(genDict)


damages_first = ['80.0', '23.0', '12.5', '40.0', '26.5', '26.5', '100.0', '40.0', '40.0', '23.0']
damages_last = ['80.0', '13.5', '6.25', '35.0', '23.0', '23.0', '45.0', '35.0', '35.0', '13.5']
dmg_distances_first = ['0.0', '6.0', '14.0', '40.0', '11.0', '11.0', '10.0', '40.0', '40.0', '6.0']
dmg_distances_last = ['165.0', '37.0', '21.0', '80.0', '35.0', '35.0', '35.0', '80.0', '80.0', '37.0']
drag = ['0.0025', '0.0025', '0.007', '0.0025', '0.002', '0.002', '0.0025', '0.0025', '0.0025', '0.0025']
firemode_Automatic = ['0.0', '1.0', '0.0', '0.0', '1.0', '1.0', '0.0', '0.0', '0.0', '1.0']
firemode_Semi-Automatic = ['0.0', '0.0', '0.0', '1.0', '0.0', '0.0', '0.0', '1.0', '1.0', '0.0']
hiprecoildec = ['4.5', '16.0', '6.0', '3.0', '6.0', '6.0', '5.0', '3.0', '5.25', '8.0']
hiprecoilright = ['1.0', '0.4', '1.2', '0.45', '0.17', '0.12', '0.8', '0.1', '0.53', '0.32']
hiprecoilup = ['5.0', '0.42', '11.0', '1.1', '0.26', '0.35', '7.0', '0.95', '1.1', '0.336']
hordispersion = ['0.0', '0.0', '2.7'